In [1]:
#pip install langchain
#pip install langdetect
#pip install gradio
#pip install pypdf
#pip install openai
#pip install chromadb

In [2]:
# from Bibliotecas ou Pacotes . Módulo import Classe 

# Importando as classes que usaremos

import os                                                                 # Biblioteca para manipular o SO
from dotenv import load_dotenv, find_dotenv                               # Carregar arquivo .env
from langchain.document_loaders import PyPDFLoader                        # Carregar o PDF
from langchain.text_splitter import RecursiveCharacterTextSplitter        # Separar as palavras
from langchain.vectorstores import Chroma                                 # Banco de Dados de Vetores
from langchain.embeddings.openai import OpenAIEmbeddings                  # Embedding das palavras
from langchain.chains import RetrievalQA                                  # Perguntas e Respostas
from langchain.chat_models import ChatOpenAI                              # Modelo utilizado
from langdetect import detect                                             # Detectar Idioma
import gradio as gr                                                       # Criar o chatbot

In [3]:
load_dotenv(find_dotenv(), override=True)

True

In [17]:
def carregar_documento(file_name):
    
    loader = PyPDFLoader(file_name)          
    pages = loader.load()                     
    text_splitter = RecursiveCharacterTextSplitter(                     
    chunk_size = 1000,                                                 
    chunk_overlap  = 0)                                                
    chunks = text_splitter.split_documents(pages)                   
    embeddings = OpenAIEmbeddings(model='text-embedding-ada-002') 
    vector_store = Chroma.from_documents(chunks, embeddings)
    #global vector_store
    print('ok')
    return file_name

def perguntas_e_respostas(query):
    llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=0.1)
    retriever = vector_store.as_retriever(search_type='similarity', search_kwargs={'k': 5}) 
    chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)   
    resposta = chain.run(query)                                                             #
    return resposta

In [1]:
with gr.Blocks() as demo:
    
    with gr.Row():
        
        # https://www.gradio.app/docs/file
        input_file = gr.File(label='Suporte apenas para PDF',  # Nome do parâmetro
                             file_types=['.pdf'],              # Arquivos Suportados
                             interactive=True,                 # Permitir que o usuário carregue um arquivo
                             scale=2)                          # Escala em comparação com outro componente da linha
        
        # https://www.gradio.app/docs/button
        input_button = gr.Button(value='Upar documento',       # Texto que o botão exibirá
                                 scale=1)                      # Escala em comparação com outro componente da linha
        
    with gr.Row():
        
        # https://www.gradio.app/docs/textbox
        textbox_files = gr.Textbox(label='Arquivo upado',       # Texto que o botão exibirá
                                   scale=1)                     # Escala em comparação com outro componente da linha
        
        # https://www.gradio.app/docs/button
        #erase_button  = gr.Button(value='Limpar a base de dados',    # Texto que o botão exibirá
                                  #scale=0)                           # Escala em comparação com outro componente da linha
        
    with gr.Column():
        
        # https://www.gradio.app/docs/chatbot
        chatbot = gr.Chatbot(label='Chatbot',                  # Texto que o botão exibirá
                             show_copy_button=True)            # Botão para copiar o texto
        
    with gr.Row():
        
        # https://www.gradio.app/docs/textbox
        prompt = gr.Textbox(label='Prompt',                    # Texto que o botão exibirá
                            scale = 1)                         # Escala em comparação com outro componente da linha
        
        # https://www.gradio.app/docs/button
        submit_button = gr.Button(value='Enviar',             # Texto que o botão exibirá
                                  scale = 0)                  # Escala em comparação com outro componente da linha
        
        # https://www.gradio.app/docs/clearbutton
        clear_button = gr.ClearButton(components=[prompt, chatbot],     # Componentes que serão limpos
                                      value='Apagar Histórico',         # Texto que o botão exibirá
                                      scale = 0)                        # Escala em comparação com outro componente da linha

        
    input_button.click(carregar_documento,          # O botao vai usar a função carregar_documentos
                        inputs=[input_file],         # O input será o arquivo upado
                        outputs=[textbox_files])     # O output será no textobox_files
        
    submit_button.click(perguntas_e_respostas,      #
                        inputs=[prompt],            # O imput será o que foi escrito no prompt
                        outputs=[chatbot])          # A output será no chatbox
        

demo.launch()

NameError: name 'gr' is not defined